<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/research/modeling_NiN/test_sample_NiN(BatchNorm_before_activation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/vishal-burman/PyTorch-Architectures.git

Cloning into 'PyTorch-Architectures'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 1437 (delta 77), reused 76 (delta 33), pack-reused 1303
Receiving objects: 100% (1437/1437), 8.57 MiB | 24.37 MiB/s, done.
Resolving deltas: 100% (845/845), done.


In [2]:
%cd PyTorch-Architectures/research/modeling_NiN/

/content/PyTorch-Architectures/research/modeling_NiN


In [5]:
import time

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader, Subset

from torchvision import datasets, transforms

from NiN_bn_before_activation import NiN

if torch.cuda.is_available():
  torch.backends.cudnn.deterministic = True

In [6]:
#############
# Settings
#############

RANDOM_SEED = 1
LEARNING_RATE = 0.0005
BATCH_SIZE = 256
EPOCHS = 10

# Architecture
NUM_CLASSES = 10

# Other
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
GRAYSCALE = False

In [7]:
########################
# CIFAR-10
########################

train_indices = torch.arange(0, 49000)
valid_indices = torch.arange(49000, 50000)

train_and_valid = datasets.CIFAR10(root='data',
                                   train=True,
                                   transform=transforms.ToTensor(),
                                   download=True)

train_dataset = Subset(train_and_valid, train_indices)
valid_dataset = Subset(train_and_valid, valid_indices)

test_dataset = datasets.CIFAR10(root='data',
                                train=False, 
                                transform=transforms.ToTensor())

########################
# DataLoaders
########################

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          num_workers=4,
                          shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset,
                          batch_size=BATCH_SIZE,
                          num_workers=4,
                          shuffle=False)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE,
                         num_workers=4,
                         shuffle=False)

############################
# Check dataset
############################
for images, labels in train_loader:
  print('Images Dimensions: ', images.shape)
  print('Labels Dimensions: ', labels.shape, "\n")
  break

for images, labels in valid_loader:
  print('Images Dimensions: ', images.shape)
  print('Labels Dimensions: ', labels.shape, "\n")
  break

for images, labels in test_loader:
  print('Images Dimensions: ', images.shape)
  print('Labels Dimensions: ', labels.shape, "\n")
  break

print('Length of train_loader: ', len(train_loader))
print('Length of valid_loader: ', len(valid_loader))
print('Length of test_loader: ', len(test_loader))

Extracting data/cifar-10-python.tar.gz to data




Images Dimensions:  torch.Size([256, 3, 32, 32])
Labels Dimensions:  torch.Size([256]) 


Images Dimensions:  torch.Size([256, 3, 32, 32])
Labels Dimensions:  torch.Size([256]) 

Images Dimensions:  torch.Size([256, 3, 32, 32])
Labels Dimensions:  torch.Size([256]) 

Length of train_loader:  192
Length of valid_loader:  4
Length of test_loader:  40


In [9]:
torch.manual_seed(RANDOM_SEED)

model = NiN(num_classes=NUM_CLASSES)
model.to(DEVICE)

params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('Total trainable parameters: ', params)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

Total trainable parameters:  968394


In [10]:
def compute_accuracy(model, data_loader, device):
  correct_pred, num_examples = 0, 0
  for idx, (features, targets) in enumerate(data_loader):
    features = features.to(device)
    targets = targets.to(device)

    logits, probas = model(features)
    _, predicted_labels = torch.max(probas, 1)
    correct_pred += (predicted_labels == targets).sum()
    num_examples += targets.size(0)
  return correct_pred.float()/ num_examples * 100

start_time = time.time()
for epoch in range(EPOCHS):
  model.train()
  for idx, (features, targets) in enumerate(train_loader):
    features = features.to(DEVICE)
    targets = targets.to(DEVICE)

    logits, _ = model(features)

    loss = F.cross_entropy(logits, targets)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # LOGGING
    if idx % 120 == 0:
      print('Batch: %04d/%04d || Epoch: %04d/%04d || Loss: %.3f' % (idx, len(train_loader), epoch+1, EPOCHS, loss.item()))
  
  model.eval()
  with torch.set_grad_enabled(False):
    train_accuracy = compute_accuracy(model, train_loader, DEVICE)
    valid_accuracy = compute_accuracy(model, valid_loader, DEVICE)
    print('Train Accuracy: %.2f' % (train_accuracy))
    print('Valid Accuracy: %.2f' % (valid_accuracy))
  epoch_elapsed_time = (time.time() - start_time) / 60
  print('Epoch Elapsed Time: %.2f min' % (epoch_elapsed_time))
total_elapsed_time = (time.time() - start_time) / 60
print('Total Elapsed Time: %.2f min' % (total_elapsed_time))

Batch: 0000/0192 || Epoch: 0001/0010 || Loss: 2.299
Batch: 0120/0192 || Epoch: 0001/0010 || Loss: 1.210
Train Accuracy: 54.34
Valid Accuracy: 52.50
Epoch Elapsed Time: 0.57 min
Batch: 0000/0192 || Epoch: 0002/0010 || Loss: 1.047
Batch: 0120/0192 || Epoch: 0002/0010 || Loss: 0.930
Train Accuracy: 63.80
Valid Accuracy: 63.00
Epoch Elapsed Time: 1.16 min
Batch: 0000/0192 || Epoch: 0003/0010 || Loss: 0.803
Batch: 0120/0192 || Epoch: 0003/0010 || Loss: 0.863
Train Accuracy: 68.95
Valid Accuracy: 68.90
Epoch Elapsed Time: 1.75 min
Batch: 0000/0192 || Epoch: 0004/0010 || Loss: 0.706
Batch: 0120/0192 || Epoch: 0004/0010 || Loss: 0.714
Train Accuracy: 69.32
Valid Accuracy: 67.80
Epoch Elapsed Time: 2.35 min
Batch: 0000/0192 || Epoch: 0005/0010 || Loss: 0.610
Batch: 0120/0192 || Epoch: 0005/0010 || Loss: 0.634
Train Accuracy: 75.90
Valid Accuracy: 72.90
Epoch Elapsed Time: 2.95 min
Batch: 0000/0192 || Epoch: 0006/0010 || Loss: 0.500
Batch: 0120/0192 || Epoch: 0006/0010 || Loss: 0.602
Train Accur